In [1]:
from keras.applications import vgg16
from keras.models import Model, Sequential
from keras.layers import Conv2D, Conv2DTranspose, Input, Cropping2D, add, Dropout, Reshape, Activation, Dense, Flatten
from keras.utils import plot_model
from glob import glob
import cv2
import sys
import numpy as np
sys.path.append("/../")
#sys.path.append("../../")

Using TensorFlow backend.


## Preprocessing



In [2]:
files_in = glob('../gd_train/*')[:5]
files_out = glob('../gd_imp_train/*')[:5]
inp = []
out = []
for f1 in files_in:
    image = cv2.imread(f1)
    image = cv2.resize(image,(224,224))
    inp.append(image)
for f2 in files_out:
    image = cv2.imread(f2)
    image = cv2.resize(image,(224,224))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    out.append(gray)
np.asarray(out).shape



(5, 224, 224)

In [ ]:
def FCN32(nClasses, input_height, input_width):

    assert input_height % 32 == 0
    assert input_width % 32 == 0

    img_input = Input(shape=(input_height, input_width, 3))

    model = vgg16.VGG16(
        include_top=False,
        weights='imagenet', input_tensor=img_input)
    assert isinstance(model, Model)

    o = Conv2D(
        filters=4096,
        kernel_size=(
            7,
            7),
        padding="same",
        activation="relu",
        name="fc6")(
            model.output)
    o = Dropout(rate=0.5)(o)
    o = Conv2D(
        filters=4096,
        kernel_size=(
            1,
            1),
        padding="same",
        activation="relu",
        name="fc7")(o)
    o = Dropout(rate=0.5)(o)

    o = Conv2D(filters=nClasses, kernel_size=(1, 1), padding="same", activation="relu", kernel_initializer="he_normal",
               name="score_fr")(o)

    o = Conv2DTranspose(filters=nClasses, kernel_size=(32, 32), strides=(32, 32), padding="valid", activation=None,
                        name="score2")(o)

    o = Reshape((-1, nClasses))(o)
    #o = Activation("softmax")(o)
    # extra layer
    o = Flatten()(o)
    o = Dense(units=(320*320), activation='sigmoid')(o)
    o = Reshape((320, 320))(o)
    fcn8 = Model(inputs=img_input, outputs=o)
    # mymodel.summary()
    return fcn8

In [ ]:
m = FCN32(256, 320, 320)
m.summary()
#plot_model(m, show_shapes=True, to_file='model_fcn32.png')
print(len(m.layers))

In [33]:
m.compile(
    loss='categorical_crossentropy',
    optimizer="adadelta",
    metrics=['acc'])

In [34]:
X = np.asarray(inp)
Y = np.asarray(out)
m.fit(X, Y)

ValueError: Error when checking target: expected activation_2 to have shape (102400, 25) but got array with shape (320, 320)